In [1]:
# function calling with openai

In [2]:
import os
import pandas as pd

import openai

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import dotenv
dotenv.load_dotenv()

True

In [3]:
openai.api_key = os.getenv('OPENAI_API_KEY')

models = openai.Model.list()
print([(i, m.id,) for i, m in enumerate(models["data"])])
models['data'][2]

[(0, 'whisper-1'), (1, 'babbage'), (2, 'davinci'), (3, 'text-davinci-edit-001'), (4, 'babbage-code-search-code'), (5, 'text-similarity-babbage-001'), (6, 'code-davinci-edit-001'), (7, 'text-davinci-001'), (8, 'ada'), (9, 'babbage-code-search-text'), (10, 'babbage-similarity'), (11, 'code-search-babbage-text-001'), (12, 'text-curie-001'), (13, 'gpt-3.5-turbo-16k-0613'), (14, 'code-search-babbage-code-001'), (15, 'text-ada-001'), (16, 'text-similarity-ada-001'), (17, 'curie-instruct-beta'), (18, 'gpt-3.5-turbo-0301'), (19, 'ada-code-search-code'), (20, 'ada-similarity'), (21, 'gpt-3.5-turbo'), (22, 'code-search-ada-text-001'), (23, 'text-search-ada-query-001'), (24, 'davinci-search-document'), (25, 'ada-code-search-text'), (26, 'text-search-ada-doc-001'), (27, 'davinci-instruct-beta'), (28, 'text-similarity-curie-001'), (29, 'code-search-ada-code-001'), (30, 'ada-search-query'), (31, 'text-search-davinci-query-001'), (32, 'curie-search-query'), (33, 'davinci-search-query'), (34, 'babbage

<Model model id=davinci at 0x7f9d389cebd0> JSON: {
  "created": 1649359874,
  "id": "davinci",
  "object": "model",
  "owned_by": "openai",
  "parent": null,
  "permission": [
    {
      "allow_create_engine": false,
      "allow_fine_tuning": false,
      "allow_logprobs": true,
      "allow_sampling": true,
      "allow_search_indices": false,
      "allow_view": true,
      "created": 1669066355,
      "group": null,
      "id": "modelperm-U6ZwlyAd0LyMk4rcMdz33Yc3",
      "is_blocking": false,
      "object": "model_permission",
      "organization": "*"
    }
  ],
  "root": "davinci"
}

In [4]:

scope = "playlist-modify-public"

sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope=scope,
                                                       client_id=os.getenv('SPOTIFY_CLIENT_ID'),
                                                       client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                       redirect_uri="https://druce.ai"
                                                      ))

In [5]:
# get playlist id
import pdb
def get_playlist_id(playlist_name, verbose=False):
    playlists = sp.user_playlists(os.getenv('SPOTIFY_USERNAME'))
    while playlists:
        for i, playlist in enumerate(playlists['items']):
            if playlist['name'] == playlist_name:
                if verbose:
                    print('"%s": offset %d, URI %s' % (playlist['name'], i + 1 + playlists['offset'], playlist['uri']))
                return playlist['id']

        # not found yet, get next page if there is one
        if playlists['next']:
            playlists = sp.next(playlists)
        else:
            return None

playlist_id = get_playlist_id("Reddit Prettiest Songs")
print(playlist_id)

08YFkbtTV6GBfNtjJ4PHDu


In [6]:
def get_tracks(playlist_id):

    results = sp.user_playlist(os.getenv('SPOTIFY_USERNAME'), 
                               playlist_id,
                               fields='tracks,next,name')
    tracks = results['tracks']
    
    # get tracks, paging as needed
    track_list = []
    while tracks:
        for track_item in tracks['items']:
            track_list.append(track_item['track'])
        # more pages?
        tracks = sp.next(tracks) if tracks['next'] else None
                    
    return pd.DataFrame({'artist': [track['artists'][0]['name'] for track in track_list],
                         'track': [track['name'] for track in track_list],
                         'uri': [track['uri'] for track in track_list],
                         'id': [track['id'] for track in track_list],
                         'popularity': [track['popularity'] for track in track_list],
                        })

get_tracks('08YFkbtTV6GBfNtjJ4PHDu')


,artist,track,uri,id,popularity
0,The Black Crowes,She Talks To Angels,spotify:track:5NK8jad728pj6YeqM5VJD4,5NK8jad728pj6YeqM5VJD4,69
1,The Cardigans,Lovefool,spotify:track:7aQjPecQdIuNd1sz3KCDhD,7aQjPecQdIuNd1sz3KCDhD,79
2,Carpenters,(They Long To Be) Close To You,spotify:track:50q2aUjWoTn6CJIfSPRJQA,50q2aUjWoTn6CJIfSPRJQA,64
3,Carpenters,Superstar,spotify:track:1Gv6f7m5ZNDcInxGA8PsLS,1Gv6f7m5ZNDcInxGA8PsLS,61
4,Carpenters,Yesterday Once More,spotify:track:3wM6RTAnF7IQpMFd7b9ZcL,3wM6RTAnF7IQpMFd7b9ZcL,69
...,...,...,...,...,...
1841,Mitski,Francis Forever,spotify:track:5411TEB6tlzvuF5A4oyldr,5411TEB6tlzvuF5A4oyldr,78
1842,Lady Gaga,Shallow,spotify:track:2VxeLyX666F8uXCJ0dZF8B,2VxeLyX666F8uXCJ0dZF8B,86
1843,Francisco Tárrega,"Recuerdos de la Alhambra, ift 44",spotify:track:5uqj1uRgouBQqdHzMLo2ag,5uqj1uRgouBQqdHzMLo2ag,32
1844,FKJ,IHM,spotify:track:7Mv1ZsKa0EG2KYwBF7eXm2,7Mv1ZsKa0EG2KYwBF7eXm2,42


In [7]:
system_prompt = """You are a Spotify helper AI. 
I will make requests related to Spotify and you will generate python code
to generate and answer using the functions provided"
"""

In [8]:
user_message1 = "What is the id of my playlist entitled 'Reddit Prettiest Songs'"

In [9]:
def get_response(messages):
    """take a chain of messages and get a ChatGPT repsosne"""
    response = openai.ChatCompletion.create(
        messages=messages,
        functions=[{"name": "get_playlist_id",
                    "description": "Return the Spotify playlist id based on the name of the playlist",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "playlist_name": {
                                "type": "string",
                                "description": "The name of the playlist"
                            },
                        },
                        "required": ["playlist_name"]
                    }
                   },
                   {
                "name": "get_tracks",
                       "description": "Return a dataframe of tracks in a playlist based on a Spotify playlist ID",
                       "parameters": {
                           "type": "object",
                           "properties": {
                               "playlist_id": {
                                   "type": "string",
                                   "description": "The Spotify playlist ID"
                               },
                           },
                           "required": ["playlist_id"]
                       }
                   },
                   
                  ],
        model="gpt-3.5-turbo-0613",
        temperature=0.1
    )
    return response

messages=[
    {
        "role": "system",
        "content": system_prompt,
    },
    {
        "role": "user",
        "content": user_message1,
    }
]

response=get_response(messages)
response

<OpenAIObject chat.completion id=chatcmpl-7RneA9d2KqW0OwxpY9SAKStFVjZoN at 0x7f9d19222810> JSON: {
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "message": {
        "content": null,
        "function_call": {
          "arguments": "{\n  \"playlist_name\": \"Reddit Prettiest Songs\"\n}",
          "name": "get_playlist_id"
        },
        "role": "assistant"
      }
    }
  ],
  "created": 1686860054,
  "id": "chatcmpl-7RneA9d2KqW0OwxpY9SAKStFVjZoN",
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 145,
    "total_tokens": 166
  }
}

In [10]:
fn_obj = response['choices'][0]['message']['function_call']
fn_obj

<OpenAIObject at 0x7f9d389b2860> JSON: {
  "arguments": "{\n  \"playlist_name\": \"Reddit Prettiest Songs\"\n}",
  "name": "get_playlist_id"
}

In [11]:
args = eval(fn_obj['arguments'])
args

{'playlist_name': 'Reddit Prettiest Songs'}

In [12]:
print(f"{fn_obj['name']}(**args)")
eval(f"{fn_obj['name']}(**args)")

get_playlist_id(**args)


'08YFkbtTV6GBfNtjJ4PHDu'

In [13]:
def get_response_and_eval(messages):
    "send messages to OpenAI and evaluate the returned function, return the value and the message payload"
    response=get_response(messages)
    fn_obj = response['choices'][0]['message']['function_call']
    args = eval(fn_obj['arguments'])
    return eval(f"{fn_obj['name']}(**args)"), response['choices'][0]['message']


In [14]:
user_message2 = "give me a dataframe of tracks in the Spotify playlist with ID 08YFkbtTV6GBfNtjJ4PHDu"
messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_message2,
            }
        ]

df, msg = get_response_and_eval(messages)
df

,artist,track,uri,id,popularity
0,The Black Crowes,She Talks To Angels,spotify:track:5NK8jad728pj6YeqM5VJD4,5NK8jad728pj6YeqM5VJD4,69
1,The Cardigans,Lovefool,spotify:track:7aQjPecQdIuNd1sz3KCDhD,7aQjPecQdIuNd1sz3KCDhD,79
2,Carpenters,(They Long To Be) Close To You,spotify:track:50q2aUjWoTn6CJIfSPRJQA,50q2aUjWoTn6CJIfSPRJQA,64
3,Carpenters,Superstar,spotify:track:1Gv6f7m5ZNDcInxGA8PsLS,1Gv6f7m5ZNDcInxGA8PsLS,61
4,Carpenters,Yesterday Once More,spotify:track:3wM6RTAnF7IQpMFd7b9ZcL,3wM6RTAnF7IQpMFd7b9ZcL,69
...,...,...,...,...,...
1841,Mitski,Francis Forever,spotify:track:5411TEB6tlzvuF5A4oyldr,5411TEB6tlzvuF5A4oyldr,78
1842,Lady Gaga,Shallow,spotify:track:2VxeLyX666F8uXCJ0dZF8B,2VxeLyX666F8uXCJ0dZF8B,86
1843,Francisco Tárrega,"Recuerdos de la Alhambra, ift 44",spotify:track:5uqj1uRgouBQqdHzMLo2ag,5uqj1uRgouBQqdHzMLo2ag,32
1844,FKJ,IHM,spotify:track:7Mv1ZsKa0EG2KYwBF7eXm2,7Mv1ZsKa0EG2KYwBF7eXm2,42


# Chain requests

- prompt to get a dataframe based on name
- correctly gives first function, get id based on name
- run the function, prompt with the convo so far and previous function returned X, what next?
- correctly returns the function to get dataframe based on name



In [15]:
chain_message1 = "get a dataframe of tracks in the Spotify playlist named 'Reddit Prettiest Songs'"
messages = []
messages.append({"role": "system", "content": system_prompt,})
messages.append({"role": "user", "content": chain_message1,})
chain_val, last_message = get_response_and_eval(messages)
chain_val

'08YFkbtTV6GBfNtjJ4PHDu'

In [16]:
last_message

<OpenAIObject at 0x7f9d195d1130> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"playlist_name\": \"Reddit Prettiest Songs\"\n}",
    "name": "get_playlist_id"
  },
  "role": "assistant"
}

In [17]:
messages.append(last_message)
chain_message2 = f"the previous function returned {chain_val}. What is the next step?"
messages.append({"role": "user", 'content': chain_message2})
messages

[{'role': 'system',
  'content': 'You are a Spotify helper AI. \nI will make requests related to Spotify and you will generate python code\nto generate and answer using the functions provided"\n'},
 {'role': 'user',
  'content': "get a dataframe of tracks in the Spotify playlist named 'Reddit Prettiest Songs'"},
 <OpenAIObject at 0x7f9d195d1130> JSON: {
   "content": null,
   "function_call": {
     "arguments": "{\n  \"playlist_name\": \"Reddit Prettiest Songs\"\n}",
     "name": "get_playlist_id"
   },
   "role": "assistant"
 },
 {'role': 'user',
  'content': 'the previous function returned 08YFkbtTV6GBfNtjJ4PHDu. What is the next step?'}]

In [18]:
df, last_msg = get_response_and_eval(messages)
df


,artist,track,uri,id,popularity
0,The Black Crowes,She Talks To Angels,spotify:track:5NK8jad728pj6YeqM5VJD4,5NK8jad728pj6YeqM5VJD4,69
1,The Cardigans,Lovefool,spotify:track:7aQjPecQdIuNd1sz3KCDhD,7aQjPecQdIuNd1sz3KCDhD,79
2,Carpenters,(They Long To Be) Close To You,spotify:track:50q2aUjWoTn6CJIfSPRJQA,50q2aUjWoTn6CJIfSPRJQA,64
3,Carpenters,Superstar,spotify:track:1Gv6f7m5ZNDcInxGA8PsLS,1Gv6f7m5ZNDcInxGA8PsLS,61
4,Carpenters,Yesterday Once More,spotify:track:3wM6RTAnF7IQpMFd7b9ZcL,3wM6RTAnF7IQpMFd7b9ZcL,69
...,...,...,...,...,...
1841,Mitski,Francis Forever,spotify:track:5411TEB6tlzvuF5A4oyldr,5411TEB6tlzvuF5A4oyldr,78
1842,Lady Gaga,Shallow,spotify:track:2VxeLyX666F8uXCJ0dZF8B,2VxeLyX666F8uXCJ0dZF8B,86
1843,Francisco Tárrega,"Recuerdos de la Alhambra, ift 44",spotify:track:5uqj1uRgouBQqdHzMLo2ag,5uqj1uRgouBQqdHzMLo2ag,32
1844,FKJ,IHM,spotify:track:7Mv1ZsKa0EG2KYwBF7eXm2,7Mv1ZsKa0EG2KYwBF7eXm2,42


In [20]:
def answer_the_question(question):
    messages = []
    last_val = None
    n_requests = 0
    MAX_RETRIES = 3
    # initial prompt
    messages.append({"role": "system", "content": system_prompt,})
    messages.append({"role": "user", "content": question,})
    
    # keep prompting for next step until exit condition or MAX_RETRIES
    for n_requests in range(MAX_RETRIES):
        last_val, last_msg = get_response_and_eval(messages)
        if type(last_val) == pd.DataFrame:
            break
        messages.append(last_msg)
        messages.append({"role": "user", 'content': f"the previous function returned {last_val}. What is the next step?"})
    
    return last_val if type(last_val) == pd.DataFrame else "retries exceeded"
        
answer_the_question("get a dataframe of tracks in the Spotify playlist named 'Reddit Prettiest Songs'")



,artist,track,uri,id,popularity
0,The Black Crowes,She Talks To Angels,spotify:track:5NK8jad728pj6YeqM5VJD4,5NK8jad728pj6YeqM5VJD4,69
1,The Cardigans,Lovefool,spotify:track:7aQjPecQdIuNd1sz3KCDhD,7aQjPecQdIuNd1sz3KCDhD,79
2,Carpenters,(They Long To Be) Close To You,spotify:track:50q2aUjWoTn6CJIfSPRJQA,50q2aUjWoTn6CJIfSPRJQA,64
3,Carpenters,Superstar,spotify:track:1Gv6f7m5ZNDcInxGA8PsLS,1Gv6f7m5ZNDcInxGA8PsLS,61
4,Carpenters,Yesterday Once More,spotify:track:3wM6RTAnF7IQpMFd7b9ZcL,3wM6RTAnF7IQpMFd7b9ZcL,69
...,...,...,...,...,...
1841,Mitski,Francis Forever,spotify:track:5411TEB6tlzvuF5A4oyldr,5411TEB6tlzvuF5A4oyldr,78
1842,Lady Gaga,Shallow,spotify:track:2VxeLyX666F8uXCJ0dZF8B,2VxeLyX666F8uXCJ0dZF8B,86
1843,Francisco Tárrega,"Recuerdos de la Alhambra, ift 44",spotify:track:5uqj1uRgouBQqdHzMLo2ag,5uqj1uRgouBQqdHzMLo2ag,32
1844,FKJ,IHM,spotify:track:7Mv1ZsKa0EG2KYwBF7eXm2,7Mv1ZsKa0EG2KYwBF7eXm2,42
